In [1]:
import collections

import numpy as np
import pandas as pd

import numba

### Preliminary

In [2]:
MISSING_INT = -99
INVALID_FLOAT = -99.00
NUM_CHOICES = 3

In [3]:
DATA_LABLES_SIM = [
    "Identifier",
    "Period",
    "Age_Level",
    "Lagged_Choice",
    "Experience_Land_Work",
    "Experience_Fishing_Work",
    "Type",
    "Choice",
    "Log_Systematic_Wage",
    "Period_Consumption_H",
    "Period_Consumption_L",
    "Period_Consumption_F",
    "Non_Consumption_Utility_H",
    "Non_Consumption_Utility_L",
    "Non_Consumption_Utility_F",
    "Flow_Utility_N",
    "Flow_Utility_P",
    "Flow_Utility_F",
    "Continuation_Value_H",
    "Continuation_Value_L",
    "Continuation_Value_F",
]

DATA_FORMATS_SIM = {
    key: (np.int if key in DATA_LABLES_SIM[:8] else np.float)
    for key in DATA_LABLES_SIM
}

In [4]:
model_spec = dict()
model_spec["num_periods"] = 60
model_spec["num_choices"] = 3
model_spec["delta"] = 0.98
model_spec["num_agents_sim"] = 100
model_spec["seed_sim"] = 397
model_spec["seed_emax"] = 2374
model_spec["num_draws_emax"] = 500
model_spec["benefits"] = 2.00
model_spec["mu"] = 0.6
model_spec = collections.namedtuple("model_specification", model_spec.keys())(
        **model_spec
    )

In [5]:
model_params = dict()

model_params["gamma_0s"] = [.3446, .2064]
model_params["gamma_1s"] = [.5363, .4223]

model_params["prob_old"] = 0.7
model_params["prob_young"] = 0.3
model_params["prob_type_low"] = 0.2
model_params["theta_f"] = [-0.36, -0.19] 
model_params["theta_l"] = [-0.24, -0.10]
model_params["shock_1_var"] = 0.025
model_params["shock_2_var"] = 0.0625
model_params["p_l"] = 0.75
model_params["delta"] = 0.05
model_params = collections.namedtuple("model_specification", model_params.keys())(
        **model_params
    )

In [6]:
model_spec

model_specification(num_periods=60, num_choices=3, delta=0.98, num_agents_sim=100, seed_sim=397, seed_emax=2374, num_draws_emax=500, benefits=2.0, mu=0.6)

In [7]:
model_params

model_specification(gamma_0s=[0.3446, 0.2064], gamma_1s=[0.5363, 0.4223], prob_old=0.7, prob_young=0.3, prob_type_low=0.2, theta_f=[-0.36, -0.19], theta_l=[-0.24, -0.1], shock_1_var=0.025, shock_2_var=0.0625, p_l=0.75, delta=0.05)

### Create states and indexer

In [8]:
def create_state_space(model_spec):  
    """This function creates the state space matrix
    and assignes an index value to every state"""
    
    data = []
    
    shape = (
        model_spec.num_periods,
        2,
        model_spec.num_choices,
        model_spec.num_periods,
        model_spec.num_periods,
        2,
    )
    
    indexer = np.full(shape, MISSING_INT)
    
    # Initialize index counter
    i = 0
    
    # Loop over all periods
    for period in range(model_spec.num_periods):
        
        # Loop over all age categories:
        for age_level in [0,1]:
            
            # Loop over all types:
            for type_ in [0,1]:
        
                # Define entry state in period zero:
                if period == 0:

                    indexer[period, age_level, 0, 0, 0, type_] = i
                    row = [period, age_level, 0, 0, 0, type_]

                    # Update counter
                    i += 1

                    # Add to state space matrix
                    data.append(row)

                else:

                    # Loop over all admissible values of experience
                    for exp_f in range(model_spec.num_periods):
                        
                        for exp_l in range(model_spec.num_periods):
                            
                            if exp_f + exp_l > period:
                                    continue                       

                            # Loop over all choices:
                            for choice_lagged in range(model_spec.num_choices):

                                # If individual has only worked in fishing in the past,
                                # she can only have fishing as lagged choice
                                if (choice_lagged != 2) and (
                                    exp_f == period
                                ):
                                    continue

                                # If individual has only worked the land in the past,
                                # she can only have part-time (1) as lagged choice
                                if (choice_lagged != 1) and (
                                    exp_l == period
                                ):
                                    continue

                                # If an individual has never worked in fishing,
                                # she cannot have that lagged activity
                                if (choice_lagged == 2) and (exp_f == 0):
                                    continue

                                # If an individual has never worked the land,
                                # she cannot have that lagged activity
                                if (choice_lagged == 1) and (exp_l == 0):
                                    continue

                                # If an individual has always been working,
                                # she cannot have home (0) as lagged choice
                                if (choice_lagged == 0) and (
                                    exp_f + exp_l == period
                                ):
                                    continue


                                # Check for duplicate states:
                                if (
                                    indexer[
                                        period, age_level, choice_lagged, exp_l, exp_f, type_
                                    ] != MISSING_INT
                                ):
                                    continue

                                # Record index of currently reached admissible state
                                # space point
                                indexer[
                                    period, age_level, choice_lagged, exp_l, exp_f, type_
                                ] = i

                                # Update count
                                i += 1

                                # Add to state space matrix
                                row = [period, age_level, choice_lagged, exp_l, exp_f, type_]
                                data.append(row)
    
        states = np.array(data)
    
    return states, indexer
    
    

In [9]:
states, indexer = create_state_space(model_spec)

In [10]:
%timeit create_state_space(model_spec)

5.22 s ± 64.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
states[np.where(np.logical_and(states[:, 0]>=0, states[:, 0]<=2))].shape[0]

52

In [12]:
indexer[0,0,0,0,0,0]

0

In [13]:
states

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1],
       [ 0,  1,  0,  0,  0,  0],
       ...,
       [59,  1,  1,  1, 58,  1],
       [59,  1,  2,  1, 58,  1],
       [59,  1,  2,  0, 59,  1]])

### Auxiliary components

In [14]:
def draw_disturbances(seed, num_periods, num_draws, model_params):
    """Creates desired number of draws of a multivariate standard normal
    distribution.

    """
    np.random.seed(seed)

    # Input parameters of the distribution
    mean = [0, 0, 0]
    shocks_cov_matrix = np.zeros((3, 3), float)
    np.fill_diagonal(shocks_cov_matrix, [0, model_params.shock_1_var, model_params.shock_2_var])

    # Create draws from the standard normal distribution
    draws = np.random.multivariate_normal(
        mean, shocks_cov_matrix, (num_periods, num_draws)
    )

    return draws

In [15]:
def calculate_utility_components(
    model_params, model_spec, states
):
    """Calculate utility components for all choices given state, period, and shocks.
    """
    log_wage_systematic = calculate_log_wage_systematic(
        model_params, states
    )

    non_consumption_utility = calculate_non_consumption_utility(
        model_params, model_spec, states
    )

    return log_wage_systematic, non_consumption_utility


def calculate_log_wage_systematic(model_params, states):
    """Calculate systematic wages, i.e., wages net of shock, for all states."""

    exp_l, exp_f = states[:, 3], states[:, 4]

    # Construct wage components
    gamma_0s = np.array(model_params.gamma_0s)[states[:, 1]]
    gamma_1s = np.array(model_params.gamma_1s)[states[:, 1]]


    period_exp_sum = exp_l * model_params.p_l + exp_f

    depreciation = 1 - model_params.delta

    # Calculate wage in the given state
    period_exp_total = period_exp_sum * depreciation + 1
    returns_to_exp = gamma_1s * np.log(period_exp_total)
    log_wage_systematic = gamma_0s + returns_to_exp

    return log_wage_systematic


def calculate_non_consumption_utility(model_params, model_spec, states):
    """Calculate non-pecuniary utility contribution."""

    non_consumption_utility = np.full(
        (states.shape[0], NUM_CHOICES), [0.00] * NUM_CHOICES
    )

    # Type contribution
    for i in [0,1]:
        non_consumption_utility[np.where(states[:, 5] == i)] += [
            0,
            model_params.theta_l[i],
            model_params.theta_f[i],
        ]

    non_consumption_utility = np.exp(non_consumption_utility)

    return non_consumption_utility

In [16]:
log_wage_systematic, non_consumption_utilities = calculate_utility_components(
    model_params, model_spec, states,
)

### Backward induction

In [17]:
log_wage_systematic

array([0.3446    , 0.3446    , 0.2064    , ..., 1.91238912, 1.91238912,
       1.91415083])

In [18]:
attrs_spec = ["seed_emax", "num_periods", "num_draws_emax"]
draws_emax = draw_disturbances(
    *[getattr(model_spec, attr) for attr in attrs_spec], model_params
)

In [19]:
def backward_induction(
    model_spec, 
    model_params, 
    states, 
    indexer, 
    log_wage_systematic, 
    non_consumption_utilities, 
    draws,
):
    """This function performs the backward induction and returns
    the model solution"""
    
    emaxs = np.zeros((states.shape[0], model_spec.num_choices + 1))
    
    
    # Loop backwards over all periods:
    for period in reversed(range(model_spec.num_periods)):
        
        # Extract period information
        # States
        states_period = states[np.where(states[:, 0] == period)]
        # Period rewards
        log_wage_systematic_period = log_wage_systematic[states[:, 0] == period]
        non_consumption_utilities_period = non_consumption_utilities[
            states[:, 0] == period
        ]
        
        # Continuation value calculation not performed for last period
        # since continuation values are known to be zero
        if period == model_spec.num_periods - 1:
            pass
        
        else:
            
            # Fill first block of elements in emaxs for the current period
            # corresponding to the continuation values
            emaxs = get_continuation_values(
                states_period,
                indexer,
                emaxs,
            )
            
        # Extract current period information for current loop calculation
        emaxs_period = emaxs[np.where(states[:, 0] == period)]
            
        # Calculate emax for current period reached by the loop
        emax_period = construct_emax(
            model_spec.delta,
            log_wage_systematic_period,
            non_consumption_utilities_period,
            draws[period],
            emaxs_period[:, :3],
            model_spec.mu,
            model_spec.benefits,
        )
        
        emaxs_period[:, 3] = emax_period
        emaxs[np.where(states[:, 0] == period)] = emaxs_period
        
    return emaxs

In [20]:
def get_continuation_values(states_subset, indexer, emaxs):
    """Obtain continuation values for each of the choices at each state
    of the period currently reached by the parent loop."""
    
    for i in range(states_subset.shape[0]):
    
        # Unpack parent state and get index
        (
            period, age_level, choice_lagged, exp_l, exp_f, type_
        ) = states_subset[i]

        k_parent = indexer[period, age_level, choice_lagged, exp_l, exp_f, type_]

        # Choice home
        k_0 = indexer[period + 1, age_level, 0, exp_l, exp_f, type_]

        # Choice land
        k_1 = indexer[period + 1, age_level, 1, exp_l + 1, exp_f, type_]

        # Choice fish
        k_2 = indexer[period + 1, age_level, 2, exp_l, exp_f + 1, type_]

        # Get emax
        emaxs[k_parent, 0] = emaxs[k_0, 3]
        emaxs[k_parent, 1] = emaxs[k_1, 3]
        emaxs[k_parent, 2] = emaxs[k_2, 3]
    
    return emaxs
    

In [21]:
@numba.njit
def _get_max_aggregated_utilities(
    delta,
    log_wage_systematic,
    non_consumption_utilities,
    draws,
    emaxs,
    mu,
    benefits,
):
    current_max_value_function = INVALID_FLOAT

    for j in range(NUM_CHOICES):

        wage = np.exp(log_wage_systematic + draws[j])

        if j == 0:
            consumption_utility = benefits ** mu / mu
        else:
            consumption_utility = (wage) ** mu / mu

        value_function_choice = (
            consumption_utility * non_consumption_utilities[j] + delta * emaxs[j]
        )

        if value_function_choice > current_max_value_function:
            current_max_value_function = value_function_choice

    return current_max_value_function


@numba.guvectorize(
    ["f8, f8, f8[:], f8[:, :], f8[:], f8, f8, f8[:]"],
    "(), (), (n_choices), (n_draws, n_choices), (n_choices), (), () -> ()",
    nopython=True,
    target="parallel",
)
def construct_emax(
    delta,
    log_wage_systematic,
    non_consumption_utilities,
    draws,
    emaxs,
    mu,
    benefits,
    emax,
):
    """Simulate expected maximum utility for a given distribution of the unobservables.

    The function calculates the maximum expected value function over the distribution of
    the error term at each state space point in the period currently reached by the
    parent loop. The expectation calculation is performed via `Monte Carlo
    integration`. The goal is to approximate an integral by evaluating the integrand at
    randomly chosen points. In this setting, one wants to approximate the expected
    maximum utility of a given state.

    """
    num_draws = draws.shape[0]

    emax[0] = 0.0

    for i in range(num_draws):
        max_total_utility = _get_max_aggregated_utilities(
            delta,
            log_wage_systematic,
            non_consumption_utilities,
            draws[i],
            emaxs,
            mu,
            benefits,
        )

        emax[0] += max_total_utility

    emax[0] /= num_draws

In [22]:
emaxs = backward_induction(model_spec, model_params, states, indexer, log_wage_systematic, non_consumption_utilities, draws_emax)

In [23]:
def pyth_simulate(
    model_params,
    model_spec,
    states,
    indexer,
    emaxs,
):
    """Simulate agent experiences."""

    # Draw random initial conditions
    np.random.seed(model_spec.seed_sim)
    age_level = np.random.choice(
        [0, 1], model_spec.num_agents_sim, p=[model_params.prob_young, model_params.prob_old]
    )

    # Draw random type
    type_ = np.random.choice(
        [0, 1],
        model_spec.num_agents_sim,
        p=[model_params.prob_type_low, 1-model_params.prob_type_low],
    )

    # Draw shocks
    attrs_spec = ["seed_sim", "num_periods", "num_agents_sim"]
    draws_sim = draw_disturbances(
        *[getattr(model_spec, attr) for attr in attrs_spec], model_params
    )

    # Calculate utility components
    log_wage_systematic, non_consumption_utilities = calculate_utility_components(
        model_params, model_spec, states
    )
    
    # Determine initial states according to initial conditions
    initial_states = pd.DataFrame(
        np.column_stack(
            (
                np.arange(model_spec.num_agents_sim),
                np.zeros(model_spec.num_agents_sim),
                age_level,
                np.zeros((model_spec.num_agents_sim, 3)),
                type_,
            )
        ),
        columns=DATA_LABLES_SIM[:7],
    ).astype(np.int)
    
    data = []
    
    # Loop over all periods
    for period in range(model_spec.num_periods):
        
        if period == 0:
            current_states = initial_states.to_numpy()
        else:
            current_states = current_states

        idx = indexer[
            current_states[:, 1],
            current_states[:, 2],
            current_states[:, 3],
            current_states[:, 4],
            current_states[:, 5],
            current_states[:, 6],
        ]

        # Extract corresponding utilities
        current_log_wage_systematic = log_wage_systematic[idx]
        current_non_consumption_utilities = non_consumption_utilities[idx]

        current_wages = np.exp(
            current_log_wage_systematic.reshape(-1, 1)
            + draws_sim[period, current_states[:, 0]]
        )

        current_wages[:, 0] = (
            model_spec.benefits
        )

        # Calculate total values for all choices
        flow_utilities = np.full((current_states.shape[0], 3), np.nan)
        flow_utilities = (
            current_wages ** model_spec.mu / model_spec.mu
        ) * current_non_consumption_utilities
        

        # Extract continuation values for all choices
        continuation_values = emaxs[idx, :3]

        value_functions = flow_utilities + model_spec.delta * continuation_values

        # Determine choice as option with highest choice specific value function
        choice = np.argmax(value_functions, axis=1)

        # Record period experiences
        rows = np.column_stack(
            (
                current_states.copy(),
                choice,
                current_log_wage_systematic,
                current_wages,
                current_non_consumption_utilities,
                flow_utilities,
                continuation_values,
            )
        )

        data.append(rows)

        # Update current states according to choice
        current_states[:, 1] += 1
        current_states[:, 3] = choice
        current_states[:, 4] = np.where(
            choice == 1, current_states[:, 4] + 1, current_states[:, 4]
        )
        current_states[:, 4] = np.where(
            choice == 2, current_states[:, 5] + 1, current_states[:, 4]
        )

    dataset = pd.DataFrame(np.vstack(data), columns=DATA_LABLES_SIM).astype(
        DATA_FORMATS_SIM
    )
    return dataset

In [24]:
data = pyth_simulate(model_params, model_spec, states, indexer, emaxs)

In [25]:
data.sort_values(by = ['Identifier', 'Period']).head(10)

,Identifier,Period,Age_Level,Lagged_Choice,Experience_Land_Work,Experience_Fishing_Work,Type,Choice,Log_Systematic_Wage,Period_Consumption_H,...,Period_Consumption_F,Non_Consumption_Utility_H,Non_Consumption_Utility_L,Non_Consumption_Utility_F,Flow_Utility_N,Flow_Utility_P,Flow_Utility_F,Continuation_Value_H,Continuation_Value_L,Continuation_Value_F
0,0,0,0,0,0,0,1,2,0.344600,2.0,...,1.283672,1.0,0.904837,0.826959,2.526194,1.803850,1.601051,165.707281,168.546731,169.441251
100,0,1,0,2,1,0,1,2,1.888792,2.0,...,8.149125,1.0,0.904837,0.826959,2.526194,4.583417,4.852878,0.000000,0.000000,0.000000
200,0,2,0,2,1,0,1,1,1.888792,2.0,...,6.488175,1.0,0.904837,0.826959,2.526194,4.584772,4.232591,0.000000,0.000000,0.000000
300,0,3,0,1,2,0,1,2,0.819671,2.0,...,3.113387,1.0,0.904837,0.826959,2.526194,2.687539,2.724411,164.743192,167.193464,167.982627
400,0,4,0,2,1,0,1,1,1.888792,2.0,...,7.196141,1.0,0.904837,0.826959,2.526194,5.155729,4.503940,0.000000,0.000000,0.000000
500,0,5,0,1,2,0,1,2,0.819671,2.0,...,2.337187,1.0,0.904837,0.826959,2.526194,2.410177,2.293765,160.314110,162.743365,163.525565
600,0,6,0,2,1,0,1,1,1.888792,2.0,...,5.990039,1.0,0.904837,0.826959,2.526194,4.335792,4.034508,0.000000,0.000000,0.000000
700,0,7,0,1,2,0,1,2,0.819671,2.0,...,3.917534,1.0,0.904837,0.826959,2.526194,2.603377,3.127091,155.750172,158.161724,158.938301
800,0,8,0,2,1,0,1,1,1.888792,2.0,...,5.289665,1.0,0.904837,0.826959,2.526194,4.080890,3.744464,0.000000,0.000000,0.000000
900,0,9,0,1,2,0,1,2,0.819671,2.0,...,3.249356,1.0,0.904837,0.826959,2.526194,2.790882,2.795189,151.046967,153.439728,154.209795


In [26]:
data["Choice"].value_counts()

2    2859
1    2787
0     354
Name: Choice, dtype: int64

In [27]:
data["Type"].value_counts()

1    4620
0    1380
Name: Type, dtype: int64

In [28]:
data["Age_Level"].value_counts()

1    4260
0    1740
Name: Age_Level, dtype: int64

In [29]:
type_ = np.random.choice(
    [0, 1],
    model_spec.num_agents_sim,
    p=[model_params.prob_type_low, 1-model_params.prob_type_low],
)

In [30]:
data.groupby(["Type"])["Choice"].value_counts(normalize=True)

Type  Choice
0     2         0.435507
      1         0.413768
      0         0.150725
1     2         0.488745
      1         0.479654
      0         0.031602
Name: Choice, dtype: float64

In [31]:
data.groupby(["Age_Level"])["Choice"].value_counts(normalize=True)

Age_Level  Choice
0          2         0.531609
           1         0.458046
           0         0.010345
1          1         0.467136
           2         0.453991
           0         0.078873
Name: Choice, dtype: float64

Moments
-------

In [35]:
NUM_PERIODS = 60

def get_moments(data):
    # Pre_process data frame

    # Determine the observed wage given period choice
    data["Consumption_Observed"] = np.nan
    data.loc[data["Choice"] == 1, "Consumption_Observed"] = data.loc[
        data["Choice"] == 1, "Period_Consumption_L"
    ]
    data.loc[data["Choice"] == 2, "Consumption_Observed"] = data.loc[
        data["Choice"] == 2, "Period_Consumption_F"
    ]

    # Calculate moments

    # Initialize moments dictionary
    moments = dict()

    # Store moments in groups as nested dictionary
    for group in [
        "Consumption_Distribution",
        "Choice_Probability",
    ]:
        moments[group] = dict()

    # Compute unconditional moments of the wage distribution
    info = data.groupby(["Period"])["Consumption_Observed"].describe().to_dict()

    # Save mean and standard deviation of wages for each period
    # to Wage Distribution section of the moments dictionary
    for period in range(NUM_PERIODS):
        moments["Consumption_Distribution"][period] = []
        try:
            for label in ["mean", "std"]:
                moments["Consumption_Distribution"][period].append(info[label][period])
        except KeyError:
            for i in range(2):
                moments["Consumption_Distribution"][period].append(0.0)

    # Compute unconditional moments of the choice probabilities
    info = data.groupby(["Period"])["Choice"].value_counts(normalize=True).to_dict()

    for period in range(NUM_PERIODS):
        moments["Choice_Probability"][period] = []
        for choice in range(3):
            try:
                stat = info[(period, choice)]
            except KeyError:
                stat = 0.00
            moments["Choice_Probability"][period].append(stat)

    return moments

In [36]:
moments = get_moments(data)

In [37]:
moments

{'Consumption_Distribution': {0: [1.2883485716957643, 0.2782417971016956],
  1: [7.297135154015751, 1.7961526109289416],
  2: [4.044987119190431, 2.6990758236079215],
  3: [5.270730787782107, 2.8183043290235816],
  4: [4.901816925179916, 2.8760401892662872],
  5: [4.734709435822299, 2.6840964420463815],
  6: [4.821536133056874, 3.0247858273874493],
  7: [5.158289387126076, 2.881633234897745],
  8: [4.9431418529225475, 2.708257082804161],
  9: [4.668891153026567, 2.7600741188176423],
  10: [5.116130133598642, 2.9464802070658767],
  11: [5.1543493078682845, 2.9421358540488596],
  12: [4.780335771315444, 2.8446809350883364],
  13: [4.986404764785814, 2.7377225038932256],
  14: [4.479652363274197, 2.7628279589777196],
  15: [5.7384445231629595, 3.023302762206762],
  16: [4.644442257547172, 2.739532881114021],
  17: [4.8499130814759255, 2.8216997669689508],
  18: [4.775640612612774, 2.888375183025627],
  19: [5.071992766929772, 2.8812352682737026],
  20: [5.103037852177328, 2.93909758948875